# 감성분석 -> 머신러닝

In [7]:
import pandas as pd
from glob import glob

file_lists = glob('C://python_src//pandas-data-analysis//machine_learning//data//train//neg//*.txt')
pd_lists = []

for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 0
        }

    df = pd.DataFrame([data])
    pd_lists.append(df)

train_neg_df = pd.concat(pd_lists, ignore_index=True)
train_neg_df.head()

,review,target
0,Story of a man who has unnatural feelings for ...,0
1,Airport '77 starts as a brand new luxury 747 p...,0
2,This film lacked something I couldn't put my f...,0
3,"Sorry everyone,,, I know this is supposed to b...",0
4,When I was little my parents took me along to ...,0


In [8]:
import pandas as pd
from glob import glob

file_lists = glob('C://python_src//pandas-data-analysis//machine_learning//data//train//pos//*.txt')
pd_lists = []

for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 1
        }

    df = pd.DataFrame([data])
    pd_lists.append(df)

train_pos_df = pd.concat(pd_lists, ignore_index=True)
train_pos_df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [9]:
train_df = pd.concat( [train_pos_df, train_neg_df], ignore_index=True )
train_df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [11]:
train_df.to_csv("movie_data.csv", index=False, encoding='utf-8')

In [13]:
df = pd.read_csv('movie_data.csv')
df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


## BOW(Bag of Words) 모델
<!-- https://www.ibm.com/kr-ko/think/topics/bag-of-words -->
- 문자를 숫자벡터 
- 단어의 등장 횟수를 카운트
---
- 전체 훈련데이터에서 모든 고유한 단어(토큰)로 어휘 사전을 만들음
- 각 문서(review data)를 사전을 기준으로 벡터화 -> n번째 단어가 문서에 3번 나오면, 벡터의 n번째 값이 3이 됨.
> - 문서1: "나는 영화가 좋다." <br>
> - 문서2: "나는 영화가 싫다." <br>
> 사전: ('나는':0 '영화가':1 '좋다':2 '싫다':3) <br>
- 벡터화는 사전의 크기만큼 모든 문장의 길이를 동일하게 함
> - 문서 1벡터: 토큰 [0,1,2] <br> -> 벡터화[1,1,1,0]
> - 문서 2벡터: 토큰 [0,1,3] <br> -> [1,1,0,1]

In [14]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
    "The sun is shining",
    "The weather is sweet"
])

bag = count.fit_transform(docs)

In [16]:
count.vocabulary_

{'the': 4, 'sun': 2, 'is': 0, 'shining': 1, 'weather': 5, 'sweet': 3}

## tf(t,d)
- 문서 d에 등장한 단어 t의 횟수를 tf(t,d)
- Bow를 보완하면서 좀 더 정교한 텍스트 벡터화 방식 TF-IDF(Term Frequency - Inverse Document Frequency)
    - TF: 특정 문서에서 자주 등장하는 단어
    - IDF: 전체 문서에서 드물게 등장하는 단어
- 특정 문서에서 자주 등장하지만, 전체 문장에서 드물게 등장하는 단어에 높은 가중치 부여 -> 그 문장을 잘 대표하는 핵심 단어를 찾는다
- `TF(t,d)`: `단어 t가 문장 d에 나타난 횟수 / 문서 d의 모든 단어 수` 
- `IDF(t,D)`: `log( 총 문서 수[D] / 단어 t를 포함한 문서 수 df(t ) )` (why? _log_ 단어의 희귀성을 너무 과하게 반영하지 않도록 스케일링) 
- 분모에 +1(사이킷런의 경우): 분모가 0이 되는 것 방지
- `log(1+[D]/1+df(f))`
> `TF-IDF(t,d,D) = TF(t,d) x IDF(t,D)` 

---


- 예시 <br>
        "나는"
    - TF: 리뷰에 3번 나옴 (가중치 높음)
    - IDF: 전체 10,000개 리뷰 중 9,000개 문장에서 나옴 (매우 낮음)
    - TF-IDF: 높음 x 매우낮음 = 낮음(중요도가 낮다) <br>
    <br>
    
    "명작"
    - TF: 리뷰에 2번 나옴 (가중치 높음)
    - IDF: 전체 10,000개 리뷰 중 50개 문장에서 나옴 (매우 높음)
    - TF-IDF: 높음 x 매우높음 = 높음(핵심 단어)


In [ ]:
print('gdd')